# Advanced SQL

## Success Criteria 

Today I will feel successful if I can...

- Connect to a database from within a python program and run queries
- Understand psycopg2's cursors and commits
- Explain SQL Injection and when it can occur 
- Generate dynamic queries
- Explain the concept Database Normalization
- Feel comfortable with the following concepts: Primary keys, foriegn keys, and the ERD


## Agenda
Part 1: Work with Combining SQl and Python
- use psycopg2 library 
- create connections to our databases stored in our docker container
- visualize outputs of our SQL queries

Part 2: RDBMS the why and how
- Reintroduce RDBMS
- Note aspects of the structure
- Go through Database Normalization
- Order of Operations for SQL
- Converse about query execution plan

# Part 1 Combining SQL and Python

Often you will find yourself working with data that are only accessable through SQL.  However, your machine-learning capabilities are built in Python.  To resolve this issue, we can simply set up a connection from Python to the SQL database to bring the data to us.

## Why do we care?

- SQL-based databases are extremely common in almost all industry environments
- Can leverage the benefit of SQL's structure and scalability, while maintaining the flexibility of Python
- Very useful for scaled data pipelines, pre-cleaning, data exploration
- Allows for dynamic query generation and hence automations

## psycopg2

- A Python library that allows for connections with PostgresSQL databases to easily query and retrieve data for analysis.
- [Documentation--Includes Installation Instructions](http://initd.org/psycopg/docs/install.html)
- In addition to what's listed in the documentation, if you have the anaconda distribution of Python 
- There are similar packages for other flavors of SQL that work much the same way

To install using anaconda try \
`conda update all`\
`conda install psycopg2`\
If that does not work for your environment try:\
`pip install psycopg2-binary`



## General Workflow

1. Establish connection to Postgres database using psycopg2
2. Create a cursor
3. Use the cursor to execute SQL queries and retrieve data
4. Commit SQL actions
4. Close the cursor and connection

# Walkthrough 1: Creating a database from Python

In [19]:
#before starting with this notebook, make sure you set your docker path. If running this locally, just use your local path.

#/home/data (from docker run command)
# Galvanize/lectures/sql-advanced' is the remaining path for me personally to get to this lecture material 
#your path is based on where you started your container and your computer file structures 
docker_path = '/home/data/Desktop/dsi/notes/sql-advance'


In [2]:
ls

img/                        psycopg2_exercise.ipynb
logins_data/                sql-advanced-lecture.ipynb
playgolf.csv


### Connect to the database
- Connections must be established using an existing database, username, database IP/URL, and maybe passwords
- If you need to create a database, you can first connect to Postgres using the dbname 'postgres' to initialize

In [3]:
import psycopg2

#first thing we need to do is establish a connection to our database
# remember our docker contianer info...
# docker run --name pgserv -d -p 5432:5432 -v "$PWD":/home/data -e POSTGRES_PASSWORD='galvanize' skylarenglish/galvanize:galv_db
conn = psycopg2.connect(dbname='dvdrental', \
                        host='localhost', \
                        port = 5432, \
                        user="postgres", \
                        password="galvanize")

In [4]:
type(conn)

psycopg2.extensions.connection

### Instantiate the Cursor

- A cursor is a control structure that enables traversal over the records in a database
- Executes and fetches data
- When the cursor points at the resulting output of a query, it can only read each observation once.  If you choose to see a previously read observation, you must rerun the query. 
- Can be closed without closing the connection

In [5]:
cur = conn.cursor()

### Take a look at dvdrentals really quick

In [6]:
query = '''
        SELECT 
            first_name,
            last_name, 
            email
        FROM customer
        LIMIT 10;
        '''
cur.execute(query)

In [7]:
cur.fetchall()

[('Jared', 'Ely', 'jared.ely@sakilacustomer.org'),
 ('Mary', 'Smith', 'mary.smith@sakilacustomer.org'),
 ('Patricia', 'Johnson', 'patricia.johnson@sakilacustomer.org'),
 ('Linda', 'Williams', 'linda.williams@sakilacustomer.org'),
 ('Barbara', 'Jones', 'barbara.jones@sakilacustomer.org'),
 ('Elizabeth', 'Brown', 'elizabeth.brown@sakilacustomer.org'),
 ('Jennifer', 'Davis', 'jennifer.davis@sakilacustomer.org'),
 ('Maria', 'Miller', 'maria.miller@sakilacustomer.org'),
 ('Susan', 'Wilson', 'susan.wilson@sakilacustomer.org'),
 ('Margaret', 'Moore', 'margaret.moore@sakilacustomer.org')]

In [8]:
#I can't view them again until I rerun the querry.... resetting the cursor 
cur.fetchall()

[]

### Disconnect from the cursor and database
- Cursors and Connections must be closed using .close() or else Postgres will lock certain operation on the database/tables to connection is severed. 

In [9]:
cur.close() # This is optional
conn.close() # Closing the connection also closes all cursors

###  Create a database
- If you need to create a database first instead of checking out an already existing one, you can first connect to Postgres using the dbname 'postgres' to initialize 

In [10]:
conn = psycopg2.connect(dbname='postgres', \
                        host='localhost', \
                        port = 5432, \
                        user="postgres", \
                        password="galvanize")

cur = conn.cursor()

### Commits

- Data changes are not actually stored until you choose to commit
- You can choose to have automatic commit by using ` autocommit = True`
- When connecting directly to the Postgres Server to initiate server level commands such as creating a database, you must use the `autocommit = True` option since Postgres does not have "temporary" transactions at the database level

In [11]:
conn.autocommit = True

In [12]:
#creating a new Database! notice we want to avoid errors so we drop it first if we accidentally already ran this cell

cur.execute('DROP DATABASE IF EXISTS temp;')
cur.execute('CREATE DATABASE temp;')
#temp database is created but is empty.
#if you check DBeaver it will be there! kidna cool huh?

In [20]:
#right now our connection is to "postgres" to create this database 
#close the connection and let's connect to "temp"
conn.close()

# Walkthrough 2: Using the new database

### Connect to our database

In [14]:
conn

<connection object at 0x7fb10049e470; dsn: 'user=postgres password=xxx dbname=postgres host=localhost port=5432', closed: 1>

In [21]:
# This time we are connecting to our new database "temp"

conn = psycopg2.connect(dbname='temp', host='localhost', port = 5432, user="postgres", password="galvanize")
cur = conn.cursor()

In [22]:
def connect_to_db(dbname, host, port, user, password=None):
    conn = psycopg2.connect(dbname = dbname, host = host, port = port, user = user, password = password)
    cur = conn.cursor()

### Create a new table

In [23]:
#make a table full of longin data... 
#Setting each column to the correct datatype! 
# why do we need to set this up? (what kind of data are we going to populate with this table?)
query = '''
        CREATE TABLE logins (
            userid integer, 
            tmstmp timestamp, 
            type varchar(10)
        );
        '''
cur.execute(query)

### Insert data into new table

In [24]:
# we added csv data into our new table called logins and now we should be able to take a peek 
query = f'''
        COPY logins 
        FROM '{docker_path}/logins_data/logins01.csv' 
        DELIMITER ',' 
        CSV;
        '''
print(query)
cur.execute(query)


        COPY logins 
        FROM '/home/data/Desktop/dsi/notes/sql-advance/logins_data/logins01.csv' 
        DELIMITER ',' 
        CSV;
        


### Run a query to get 30 records from our data

In [36]:
query = '''
        SELECT *
        FROM logins
        LIMIT 30;
        '''
cur.execute(query)

### Lets look at our data one line at a time

In [26]:
#How do I look at one of the records? 
cur.fetchone()

(579, datetime.datetime(2013, 11, 20, 3, 20, 6), 'mobile')

### Many lines at a time

In [27]:
#fetchmany(n) to get n rows (notice we don't get the first one anymore )
cur.fetchmany(10)

[(823, datetime.datetime(2013, 11, 20, 3, 20, 49), 'web'),
 (953, datetime.datetime(2013, 11, 20, 3, 28, 49), 'web'),
 (612, datetime.datetime(2013, 11, 20, 3, 36, 55), 'web'),
 (269, datetime.datetime(2013, 11, 20, 3, 43, 13), 'web'),
 (799, datetime.datetime(2013, 11, 20, 3, 56, 55), 'web'),
 (890, datetime.datetime(2013, 11, 20, 4, 2, 33), 'mobile'),
 (330, datetime.datetime(2013, 11, 20, 4, 54, 59), 'mobile'),
 (628, datetime.datetime(2013, 11, 20, 4, 57, 22), 'mobile'),
 (398, datetime.datetime(2013, 11, 20, 5, 3, 19), 'mobile'),
 (482, datetime.datetime(2013, 11, 20, 5, 4, 43), 'mobile')]

### Skip some lines

In [28]:
cur.scroll(4)

### Or everything at once

In [29]:
#How do I grab all remaining rows at least? (set to varibale results)
cur.fetchall()

[(274, datetime.datetime(2013, 11, 20, 5, 43, 8), 'mobile'),
 (581, datetime.datetime(2013, 11, 20, 5, 47, 10), 'web'),
 (417, datetime.datetime(2013, 11, 20, 5, 54, 37), 'mobile'),
 (185, datetime.datetime(2013, 11, 20, 5, 56, 22), 'mobile'),
 (371, datetime.datetime(2013, 11, 20, 5, 58, 35), 'mobile'),
 (133, datetime.datetime(2013, 11, 20, 5, 59, 7), 'web'),
 (621, datetime.datetime(2013, 11, 20, 6, 1, 46), 'web'),
 (306, datetime.datetime(2013, 11, 20, 6, 3, 23), 'mobile'),
 (509, datetime.datetime(2013, 11, 20, 6, 4, 43), 'web'),
 (505, datetime.datetime(2013, 11, 20, 6, 9, 52), 'web'),
 (678, datetime.datetime(2013, 11, 20, 6, 34, 18), 'web'),
 (889, datetime.datetime(2013, 11, 20, 6, 36, 32), 'mobile'),
 (202, datetime.datetime(2013, 11, 20, 6, 43, 33), 'mobile'),
 (614, datetime.datetime(2013, 11, 20, 6, 47, 55), 'mobile'),
 (882, datetime.datetime(2013, 11, 20, 6, 49), 'mobile')]

In [37]:
result = list(cur)

In [38]:
result

[(579, datetime.datetime(2013, 11, 20, 3, 20, 6), 'mobile'),
 (823, datetime.datetime(2013, 11, 20, 3, 20, 49), 'web'),
 (953, datetime.datetime(2013, 11, 20, 3, 28, 49), 'web'),
 (612, datetime.datetime(2013, 11, 20, 3, 36, 55), 'web'),
 (269, datetime.datetime(2013, 11, 20, 3, 43, 13), 'web'),
 (799, datetime.datetime(2013, 11, 20, 3, 56, 55), 'web'),
 (890, datetime.datetime(2013, 11, 20, 4, 2, 33), 'mobile'),
 (330, datetime.datetime(2013, 11, 20, 4, 54, 59), 'mobile'),
 (628, datetime.datetime(2013, 11, 20, 4, 57, 22), 'mobile'),
 (398, datetime.datetime(2013, 11, 20, 5, 3, 19), 'mobile'),
 (482, datetime.datetime(2013, 11, 20, 5, 4, 43), 'mobile'),
 (581, datetime.datetime(2013, 11, 20, 5, 12, 3), 'mobile'),
 (370, datetime.datetime(2013, 11, 20, 5, 26, 46), 'mobile'),
 (230, datetime.datetime(2013, 11, 20, 5, 28, 29), 'web'),
 (596, datetime.datetime(2013, 11, 20, 5, 28, 36), 'web'),
 (274, datetime.datetime(2013, 11, 20, 5, 43, 8), 'mobile'),
 (581, datetime.datetime(2013, 11, 

In [40]:
print(len(result))

30


### You can even iterate over the cursor

In [41]:
#since we stored our query in a variable, I can re run it easily here
cur.execute(query)
for record in cur:
    print(f"On {record[1]}: user {record[1]} logged in via {record[2]}")

On 2013-11-20 03:20:06: user 2013-11-20 03:20:06 logged in via mobile
On 2013-11-20 03:20:49: user 2013-11-20 03:20:49 logged in via web
On 2013-11-20 03:28:49: user 2013-11-20 03:28:49 logged in via web
On 2013-11-20 03:36:55: user 2013-11-20 03:36:55 logged in via web
On 2013-11-20 03:43:13: user 2013-11-20 03:43:13 logged in via web
On 2013-11-20 03:56:55: user 2013-11-20 03:56:55 logged in via web
On 2013-11-20 04:02:33: user 2013-11-20 04:02:33 logged in via mobile
On 2013-11-20 04:54:59: user 2013-11-20 04:54:59 logged in via mobile
On 2013-11-20 04:57:22: user 2013-11-20 04:57:22 logged in via mobile
On 2013-11-20 05:03:19: user 2013-11-20 05:03:19 logged in via mobile
On 2013-11-20 05:04:43: user 2013-11-20 05:04:43 logged in via mobile
On 2013-11-20 05:12:03: user 2013-11-20 05:12:03 logged in via mobile
On 2013-11-20 05:26:46: user 2013-11-20 05:26:46 logged in via mobile
On 2013-11-20 05:28:29: user 2013-11-20 05:28:29 logged in via web
On 2013-11-20 05:28:36: user 2013-11-2

# Dynamic Queries

- A Dynamic Query is a query that generates based on context.

- Sometimes, the SQL string needs to be constructed dynamically, given some input parameters.... but be careful! We will discuss the dangers of SQL injection! 

### Example

We have 8 login csv files that we need to insert into the logins table.  Instead of doing a COPY FROM query 8 times, we should utilize Python (or any future languages) to make this more efficient. 

### First lets get an idea of how many records we start with

In [42]:
cur.execute('SELECT count(*) FROM logins;')
record_count = cur.fetchone()[0]

In [43]:
record_count

10000

In [47]:
import pandas as pd

logins01 = pd.read_csv('logins_data/logins01.csv')
logins01.shape

(9999, 3)

In [44]:
type(record_count)

int

In [45]:
conn.commit()

### Create a query template and determine file path for imports


## **[WARNING: BEWARE OF SQL INJECTION](http://initd.org/psycopg/docs/usage.html)**

### What is an SQL Injection Attack?

from W3schools.com:

 - SQL injection is a code injection technique that might destroy your database.
 - SQL injection is **one of the most common** web hacking techniques.
 - SQL injection is the placement of malicious code in SQL statements, via web page input.
 
SQL injection usually occurs when you ask a user for input, like their username/userid, and instead of a name/id, the user gives you an SQL statement that you will **unknowingly run on your database.**

Here is an example:

In [48]:
date_cut = "2013-10-20; DROP TABLE logins" # The user enters a date in a field on a web form

horribly_risky = "SELECT * FROM logins WHERE tmstmp > %s;" % date_cut

print(horribly_risky)

SELECT * FROM logins WHERE tmstmp > 2013-10-20; DROP TABLE logins;


In [52]:
date_cut = "2013-10-20; DROP TABLE logins" # The user enters a date in a field on a web form

horribly_risky = f"SELECT * FROM logins WHERE tmstmp > {date_cut}s;" 

print(horribly_risky)

SELECT * FROM logins WHERE tmstmp > 2013-10-20; DROP TABLE loginss;


To avoid SQL Injections **NEVER** use + or % or .format to reformat strings to be used with .execute

In [56]:
num = 579
terribly_unsafe = "SELECT * FROM logins WHERE userid = " + str(num) + ";"
print(terribly_unsafe)


date_cut = "2013-10-20"
horribly_risky = "SELECT * FROM logins WHERE tmstmp > %s;" % date_cut
print(horribly_risky)
## Python is happy, but if num or date_cut included something malicious
## your data could be at risk

SELECT * FROM logins WHERE userid = 579;
SELECT * FROM logins WHERE tmstmp > 2013-10-20;


### Let's write these queries safely

The Python string operator % must not be used: the execute() method accepts a tuple or dictionary of values as second parameter. Never use % or + to merge values into queries:


In [59]:
date_cut = "2013-10-20; DROP TABLE logins" # The user enters a date in a field on a web form

date_cut2 = "2013-10-20"

safe_dynamic_query = "SELECT tmstmp FROM logins WHERE tmstmp > %(date_cut)s ;" 

print(safe_dynamic_query, {'date_cut':date_cut2})

cur.execute(safe_dynamic_query, {'date_cut':date_cut2})

cur.fetchmany(10)

SELECT tmstmp FROM logins WHERE tmstmp > %(date_cut)s ; {'date_cut': '2013-10-20'}


[(datetime.datetime(2013, 11, 20, 3, 20, 6),),
 (datetime.datetime(2013, 11, 20, 3, 20, 49),),
 (datetime.datetime(2013, 11, 20, 3, 28, 49),),
 (datetime.datetime(2013, 11, 20, 3, 36, 55),),
 (datetime.datetime(2013, 11, 20, 3, 43, 13),),
 (datetime.datetime(2013, 11, 20, 3, 56, 55),),
 (datetime.datetime(2013, 11, 20, 4, 2, 33),),
 (datetime.datetime(2013, 11, 20, 4, 54, 59),),
 (datetime.datetime(2013, 11, 20, 4, 57, 22),),
 (datetime.datetime(2013, 11, 20, 5, 3, 19),)]




To use .execute the corrent way, see how in the query string we used 
`%(file_path)s`\
Then in the `.execute()` method we used `{'file_path':file_path}` as a parameter


#### For more on SQL injections check out [bobby-tables.com](http://www.bobby-tables.com/)


#### Now back to our query template and determine file path for imports

### Practice safe SQL with Psycopg2

In [60]:
query = '''
        COPY logins 
        FROM %(file_path)s
        DELIMITER ','
        CSV;
        '''

In [65]:
ls 

img/                        psycopg2_exercise.ipynb
logins_data/                sql-advanced-lecture.ipynb
playgolf.csv


In [66]:
import os 

folder_path_local = 'logins_data/'

os.listdir(folder_path_local)

['logins08.csv',
 'logins06.csv',
 'logins07.csv',
 'logins05.csv',
 'logins04.csv',
 'logins01.csv',
 'logins03.csv',
 'logins02.csv']

In [67]:
# docker_path = '/home/data/Desktop/dsi/notes/sql-advance'


folder_path_docker = docker_path + '/logins_data/'
# folder_path_local = local_path + '/logins_data/'
folder_path_local = 'logins_data/'


for file_name in os.listdir(folder_path_local):
    if file_name.endswith('.csv') and file_name != 'logins01.csv':
        file_path=folder_path_docker+file_name
        cur.execute(query, {'file_path':file_path})
        print('{0} inserted into table.'.format(file_name))

logins08.csv inserted into table.
logins06.csv inserted into table.
logins07.csv inserted into table.
logins05.csv inserted into table.
logins04.csv inserted into table.
logins03.csv inserted into table.
logins02.csv inserted into table.


### Now, Let's check the total number of records we have right now.

In [69]:
print("Old record count: {}".format(record_count))

cur.execute('SELECT count(*) FROM logins;')
record_count = cur.fetchone()[0]

print("New record count: {}".format(record_count))

Old record count: 10000
New record count: 78588


### Transactions can be rolled back until they're committed

In [70]:
conn.rollback()

cur.execute('SELECT count(*) FROM logins;')
record_count = cur.fetchone()[0]

print("After rollback: {}".format(record_count))

After rollback: 10000


### Don't forget to commit your changes

In [71]:
conn.commit()

In [72]:
cur.execute('SELECT count(*) FROM logins;')
record_count = cur.fetchone()[0]

print("New record count: {}".format(record_count))

New record count: 10000


### Close your connection

In [73]:
conn.close()

### Using python `with` Statements

In [74]:

query = "SELECT count(*) FROM logins;"
with psycopg2.connect(dbname='temp', \
                      host='localhost', \
                      port = 5432, \
                      user="postgres", \
                      password="galvanize") as conn:
    with conn.cursor() as curs:
        print("Cursor inside with block: {}".format(curs))
        curs.execute(query)
        record_count = curs.fetchone()[0]
        print(record_count)
    print("Cursor outside with block: {}".format(curs))
    

Cursor inside with block: <cursor object at 0x7fb12076a050; closed: 0>
10000
Cursor outside with block: <cursor object at 0x7fb12076a050; closed: -1>


### Note that the connection is *not* closed automatically:

In [75]:
conn

<connection object at 0x7fb100f5f890; dsn: 'user=postgres password=xxx dbname=temp host=localhost port=5432', closed: 0>

In [76]:
conn.close()
conn

<connection object at 0x7fb100f5f890; dsn: 'user=postgres password=xxx dbname=temp host=localhost port=5432', closed: 1>

### Key Things to Remember about psycopg2

* Connections must be established using an existing database, username, database IP/URL, and maybe passwords
* If you have no created databases, you can connect to Postgres using the dbname 'postgres' to initialize db commands
* Data changes are not actually stored until you choose to commit. This can be done either through `conn.commit()` or setting `autocommit = True`.  Until commited, all transactions are only temporary stored.
* Autocommit = True is necessary to do database commands like CREATE DATABASE.  This is because Postgres does not have temporary transactions at the database level.
* If you ever need to build similar pipelines for other forms of database, there are libraries such PyODBC which operate very similarly.
* SQL connection databases utilizes cursors for data traversal and retrieval.  This is kind of like an iterator in Python.
* Cursor operations typically goes like the following:
    - execute a query
    - fetch rows from query result if it is a SELECT query
    - because it is iterative, previously fetched rows can only be fetched again by rerunning the query
    - close cursor through .close()
* Cursors and Connections must be closed using .close() or else Postgres will lock certain operation on the database/tables to connection is severed. 

# Part 2 RDBMS the why and how:

## Relational Database Management System (RDBMS)

#### It is a persistent data storage system
 - Schema defines the structure of a table or a database
 - Database is composed of a number of user-defined tables
 - Each table has columns (or fields) and rows (or records)
 - A column is of a certain data type such as an integer, text, or date

With a new data source, your first task is typically to understand the schema. 
This will likely take time and conversations with those that gave you access to the database or its data.



![RDBMS Diagram](img/RDBMS_Diagram.png)

#### RDBMS and SQL

 - SQL is the language used to query relational databases
 - **All RDBMS** use SQL and the syntax and keywords are the same for the most part, across systems
 - **SQL is used to interact** with RDBMS, allowing you to create tables, alter tables, insert records, update records, delete records, and query records within and across tables.
 - Even non-relational databases like **Hadoop** usually have a SQL-like interface available.

#### Database Life Cycle (DBLC)

![DBLC](img/DBLC_Diagram.jpg)

#### An inefficient way to store data...

##### A single table with records of customer purchases at an outdoor sports store.

![Inefficient Table](img/Inefficient_Table.jpg)

#### Relational Database Management Systems

 - A RDBMS is a type of database where **data is stored in multiple related tables.**
 - The tables are related through **primary** and **foreign keys**.
 - The same information as shown before in an RDBMS:


#### Primary Keys

 - Every table in a RDBMS has a **primary key**  that uniquely identifies that row
 - Each entry must have a primary key, and primary keys cannot repeat within a table
 - Primary keys are usually integers, often [GUIDs or UUIDs](https://www.guidgenerator.com/online-guid-generator.aspx), but can take other forms


#### Foreign Keys and Table Relationships

 - A **foreign key** is a column that uniquely identifies a column in another table
 - Often, a foreign key in one table is a primary key in another table
 - We can use foreign keys to join tables
 
  ![Foreign Key Diagram](img/Foreign_Key_Diagram.jpg)


#### Entity Relationship Diagram (ERD)

An Entity Relationship Diagram (ERD) represents how each forien key and primary key connects the tables. When using real production data, these diagrams can take up many pages.

![ERD](img/ERD_Diagram.jpg)

#### Database Normalization
- technique of organizing the data in the database)
    - reduce redundancy of data
    - make sure data dependancies make some logical sense
    
    
##### Store each piece of information in exactly one place
 - Details about a user (address, age) are only stored once (in a users table)
 - Any other table (eg. purchases) where this data might be relevant, only references the user_id
 
![RDBMS](img/RDBMS_Diagram.png)

##### Why is this a good thing?


#### Normal Forms (levels of normalization) 

Normalization is explained through *Normal forms*

 - 1st Normal Form: a single cell holds only a single value
     
<!-- Example: Employee information stored in one table and an employee has two phone numbers... 
Update the table to allow a different row of name, salary... for each phone number they have now each cell would have one value only
 -->

 - 2nd Normal Form: no dependencies on part of a key
 
<!--  Example: Employee ID and Department ID make up the primary key for a table... and lets say office location is a column which only depends on department id... no good... lets fix it by splitting into two tables... -->
 
 
 - 3rd Normal Form: no transitive dependencies
 
<!--  Example: Employee id determines which deparment id they are in and the department id determines the department name. Therefore we have a transitive relationship... If we split the data where any non-primary attribute is not dependent on another non-primary attribute we will improve the referential integrety of our db and elimiate duplication of data -->


![normal_forms](img/normal_forms.png)

**There are more leveles but we just need to start with the first three

#### Why Not Normalize?

Sometimes databases are not fully normalized.

 - Data structure not known/may change
 - Writing a schema/converting data is hard
 - Simple queries are important
 - Data will not be changed/integrity not important
 - Storage is cheap


![DLBC2](img/DBLC_Diagram2.jpg)
#### Data Science in the DBLC

 - Data Science Operations: querying, aggregating
 - Data Science Implementation: identifying, cleaning, pushing external data sources inside a RDBMS
 - Data Science Design: recommendations on the model, specs on operations
 
 

#### Creating a table with a schema

![Create Table](img/create_table.jpg)

#### Inserting values into a table

![insert into table](img/Insert_into_table.jpg)

#### SQL Queries for table creation / maintenance

Creating a table from query:
```sql
CREATE [TEMPORARY] TABLE table AS <SQL query>;
```
Inserting records in a table:
```sql
INSERT INTO table [(c1,c2,c3,…)] VALUES (v1,v2,v3,…);
```
Updating records:
```sql
UPDATE table SET c1=v1,c2=v2,… WHERE cX=vX;
```
Delete records:
```sql
DELETE FROM table WHERE cX=vX;
```
Change model (add, drop, modify columns):
```sql
ALTER TABLE table [DROP/ADD/ALTER] column [datatype];
```
Delete a table:
```sql
DROP TABLE table;
```

#### Order of Evaluation of a SQL SELECT Statement

1. FROM + JOIN: first the product of all tables is formed
2. WHERE: the where clause filters rows that do not meet the search condition
3. GROUP BY + (COUNT, SUM, etc): the rows are grouped using the columns in the group by clause and the aggregation functions are applied on the grouping
4. HAVING: like the WHERE clause, but can be applied after aggregation
5. SELECT: the targeted list of columns are evaluated and returned
6. DISTINCT: duplicate rows are eliminated
7. ORDER BY: the resulting rows are sorted


#### Order of Evaluation of a SQL SELECT Statement

![Order of Evaluation](img/Order_of_Evaluation.jpg)

#### Excecution Plan

When troubleshooting a query, you may want to figure out what your query is in fact doing. Most SQL systems have their own internal query optomiser, so if the query you write is not in the most efficient form, the sql system will optomize it. Sometimes still your query will fail. You can use the Execution plan to troubleshoot this.

![Exec](img/exec_plan.png)

If you are concerned about running too large a query, you can run your execution plan prior to running the query. Running to large of a query may be hard to accomplish using the datasets you find online, but can be a real concern when working with real world data.

## Lecture Objectives

#### Part 1 SQL with python (psycopg2):

- Connect to a database from within a python program and run queries
- Understand psycopg2's cursors and commits
- Generate dynamic queries

#### Part 2 RDBMS the why and how:

 - Why an RDBMS?
 - The database lifecycle (and where the data scientist fits!)
 - Primary keys, foriegn keys, and the ERD
 - Creating and maintaining tables
 - Order of Evaluation of a SQL SELECT Statement
 - Transactions
 - SQL Execution Plan

In [77]:
import sqlite3